# Pagsusuri ng Paghahabol sa Gastos

Ipinapakita ng notebook na ito kung paano gumawa ng mga ahente na gumagamit ng mga plugin upang iproseso ang mga gastusin sa paglalakbay mula sa mga lokal na larawan ng resibo, bumuo ng email para sa paghahabol sa gastos, at mag-visualize ng datos ng gastusin gamit ang pie chart. Ang mga ahente ay dinamikong pumipili ng mga function batay sa konteksto ng gawain.

Mga Hakbang:
1. Ang OCR Agent ay nagpoproseso ng lokal na larawan ng resibo at kumukuha ng datos ng gastusin sa paglalakbay.
2. Ang Email Agent ay bumubuo ng email para sa paghahabol sa gastos.

### Halimbawa ng senaryo ng gastusin sa paglalakbay:
Isipin na ikaw ay isang empleyado na naglalakbay para sa isang business meeting sa ibang lungsod. May patakaran ang iyong kumpanya na mag-reimburse ng lahat ng makatwirang gastusin na may kaugnayan sa paglalakbay. Narito ang breakdown ng mga posibleng gastusin sa paglalakbay:
- Transportasyon:
Pamasahe sa eroplano para sa round trip mula sa iyong lungsod patungo sa lungsod ng destinasyon.
Taxi o ride-hailing services papunta at pabalik sa paliparan.
Lokal na transportasyon sa lungsod ng destinasyon (tulad ng pampublikong sasakyan, rental cars, o taxi).

- Akomodasyon:
Panuluyan sa hotel para sa tatlong gabi sa isang mid-range na business hotel malapit sa lugar ng meeting.

- Pagkain:
Araw-araw na allowance para sa almusal, tanghalian, at hapunan, batay sa per diem policy ng kumpanya.

- Iba Pang Gastusin:
Bayad sa parking sa paliparan.
Bayad sa internet access sa hotel.
Mga tip o maliliit na service charge.

- Dokumentasyon:
Isinusumite mo ang lahat ng resibo (mga flight, taxi, hotel, pagkain, atbp.) at isang kumpletong ulat ng gastusin para sa reimbursement.


## I-import ang mga kinakailangang library

I-import ang mga kinakailangang library at module para sa notebook.


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## Tukuyin ang mga Modelo ng Gastos

Gumawa ng Pydantic model para sa mga indibidwal na gastos at isang klase na ExpenseFormatter upang i-convert ang query ng user sa naka-istrukturang datos ng gastos.

Ang bawat gastos ay kakatawanin sa format na:
`{'date': '07-Mar-2025', 'description': 'flight to destination', 'amount': 675.99, 'category': 'Transportation'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## Pagpapakilala ng mga Ahente - Pagbuo ng Email

Gumawa ng isang klase ng ahente para bumuo ng email na gagamitin sa pagsusumite ng claim sa gastusin.  
- Ang ahenteng ito ay gumagamit ng `kernel_function` na dekorador upang magtakda ng isang function na bumubuo ng email para sa pagsusumite ng claim sa gastusin.  
- Kinakalkula nito ang kabuuang halaga ng mga gastusin at inaayos ang mga detalye sa katawan ng email.  


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# Ahente para sa Pagkuha ng Gastos sa Paglalakbay mula sa Mga Larawan ng Resibo

Gumawa ng klase ng ahente para kunin ang mga gastos sa paglalakbay mula sa mga larawan ng resibo.
- Ang ahenteng ito ay gumagamit ng `kernel_function` na dekorador upang magtakda ng isang function na kumukuha ng mga gastos sa paglalakbay mula sa mga larawan ng resibo.
- I-convert ang larawan ng resibo sa teksto gamit ang OCR (Optical Character Recognition) at kunin ang mga kaugnay na impormasyon tulad ng petsa, deskripsyon, halaga, at kategorya.


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## Pagproseso ng Gastos

Mag-define ng asynchronous na function para iproseso ang mga gastos sa pamamagitan ng paglikha at pagrehistro ng mga kinakailangang ahente, at pagkatapos ay tawagin ang mga ito.
- Pinoproseso ng function na ito ang mga gastos sa pamamagitan ng pag-load ng mga environment variables, paglikha ng mga kinakailangang ahente, at pagrehistro sa kanila bilang mga plugin.
- Gumagawa ito ng group chat kasama ang dalawang ahente at nagpapadala ng prompt na mensahe upang makabuo ng email at pie chart batay sa datos ng gastos.
- Pinangangasiwaan nito ang anumang error na maaaring mangyari habang tinatawag ang chat at tinitiyak ang tamang paglilinis ng mga ahente.


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## Pangunahing function

Tukuyin ang pangunahing function upang linisin ang console at patakbuhin ang `process_expenses` function nang asynchronous.


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
